<a href="https://colab.research.google.com/github/ensarkaya/CS425-Group7-TermProject/blob/master/LASTVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import csv
import pandas as pd
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
path = '/content/gdrive/My Drive/webScale/100kbody.csv'
df = pd.read_csv(path)
size =100000

In [3]:
nltk.download('stopwords')
dummy =["and","or","is","are","what","where","who","whom","when","in"
            ,"a","an","of","to","for","from","by","can","i","with","its",
             "there","that","thats","all","does","do","did","the","am","using","i",
             "trying","how","be","not","doesn't","don't","didn't","won't","will",
             "my","be","should","would","as","use","must","have","but","however","at",
             "another","other","one","two","three","four","five","still","on","why","it"
              ,"how","a","to","'",".",",",";",":","?","!","-","_","1","2","3","4","5","6","7","8","9","0"]
stop = stopwords.words('english')
stop.append(dummy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
for i in range(size):
  str1 = df.loc[i,'body']
  soup = BeautifulSoup(str1)
  for a in soup.find_all('p'):
    if a.string is None:
      df.loc[i,'body']="None"  
    else:
      df.loc[i,'body']=a.string

In [0]:
df["title"] = df["title"].apply(lambda x: x.lower())
df["title"] = df["title"].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))
df["tags"] = df["tags"].apply(lambda x: x.lower())
df["tags"] = df["tags"].str.split('|').apply(lambda x: ' '.join(k for k in x if k not in dummy))
df["tags"] = df["tags"].str.split('-').apply(lambda x: ' '.join(k for k in x if k not in dummy))
#df["body"] = df["body"].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))

In [0]:
#keywords -> id's of questions
dic = {}
#questionId -> (score, answerCount, viewCount, comment_count)
values = {}

for i in range(size):
  values[df.loc[i,'id']] = (df.loc[i,'title'], df.loc[i,'score'], df.loc[i,'answer_count'], df.loc[i,'view_count'], df.loc[i,'comment_count'])
  #print("idler : ",df.loc[i,'id'])
  #print("valueler : ",values[df.loc[i,'id']])
  df.loc[i,'title']#unneccessary
  string1 = df.loc[i,'title'].split()
  string2= df.loc[i,'tags'].split()
  #string3=df.loc[i,'body'].split()
  string=string1+string2#+string3
  for j in range(len(string)):
    while len(string[j]) > 0 and (string[j][0] < 'a' or string[j][0] > 'z') and (string[j][0] < '0' or string[j][0] > '9') and string[j][0] != '#' and string[j][0] != '+':
      string[j] = string[j][1:]
    while len(string[j]) > 0 and (string[j][len(string[j])-1] < 'a' or string[j][len(string[j])-1] > 'z') and (string[j][len(string[j])-1] < '0' or string[j][len(string[j])-1] > '9') and string[j][len(string[j])-1] != '#' and string[j][len(string[j])-1] != '+':
      string[j] = string[j][:len(string[j])-1]
      
    if  string[j] != "":
      if string[j] in dic:
        dic[string[j]].append(df.loc[i,'id'])
      else:
        dic[string[j]]=[df.loc[i,'id']]


In [0]:
dic2 = {} #keywords -> keyword count
dic3 = {} #questionId -> keywords if greater than threshold
dic4 = {} #keyword -> index
dic5 = {} #questionId -> keywords as indices

threshold = 2; 
count2=0
for i in dic:
#   print(i)
  count=0
  key = i
  
  for j in dic[i]:
    count+=1
  dic2[i]=count
#   print(i," **** ", count)
  if count > threshold:
    for j in range(count):
      id = dic[i][j] 
      if id in dic3:
        dic3[id].append(i)
      else:
        dic3[id] = [i]
    if key not in dic4:
      dic4[key] = count2
      count2+=1

In [0]:
for i in dic3:
  List = []
  for j in dic3[i]:
    value = dic4[j]
#     dic3[i][j] = value
    List.append(value)
  dic5[i] = List
  
file = open("output.txt", "w")   
for i in dic5:   
#   print(i," : ",dic5[i])
#   var = i + " : " + dic5[i]
  file.write("%d" % i)
  file.write(" : ")
  for j in dic5[i]:
    file.write("%d " % j)
  file.write("\n")
file.close()

In [0]:
import random

randomNum={}
for i in range(128):
  a=random.randint(1,50000)
  b=random.randint(1,50000)
  randomNum[i]=(a,b)

In [0]:
col=len(dic5)
row=128
matrix = [[0] * col for i in range(row)]
reverseId = {}
for i in range(128):
  count=0
  for j in dic5:
    minSoFar=9999999
    for k in dic5[j]:
      hashval=(randomNum[i][0]*k + randomNum[i][1]) % 50311
      if hashval<minSoFar:
        minSoFar = hashval
    matrix[i][count]=minSoFar
    #reverse[0] -> excel 0. satırdaki Q IDsi
    reverseId[count] = j
    count +=1  

In [0]:
file2 = open("signature.txt", "w")

for i in range(128):
    for j in range(size):
      file2.write("%d" % matrix[i][j])
      file2.write(" ")
    file2.write("\n")

In [10]:
#keywords
question = input()
s = question.split(" ")
string = [x for x in s if x not in dummy]

print(string)

for j in range(len(string)):
  string[j] = string[j].lower()
  while len(string[j]) > 0 and (string[j][0] < 'a' or string[j][0] > 'z') and (string[j][0] < '0' or string[j][0] > '9') and string[j][0] != '#' and string[j][0] != '+':
      string[j] = string[j][1:]
  while len(string[j]) > 0 and (string[j][len(string[j])-1] < 'a' or string[j][len(string[j])-1] > 'z') and (string[j][len(string[j])-1] < '0' or string[j][len(string[j])-1] > '9') and string[j][len(string[j])-1] != '#' and string[j][len(string[j])-1] != '+':
      string[j] = string[j][:len(string[j])-1]
      
      
print("final string : ", string)      

if(len(string) <= 0):
  print('Question not valid.\n')
else:
    #calculating the metascore
        
    #min-hash of the keywords
    index = []
    for i in string:
      if i in dic4:
        index.append(dic4[i]);
    print("index : ", index)
    minHash = []
    for i in randomNum:
      minValue = 50311
      for j in index:
        hashValue = (randomNum[i][0]*j + randomNum[i][1]) % 50311
        if(hashValue < minValue):
          minValue = hashValue
      minHash.append(minValue)
      
    #print("minHash list : ",minHash)
    
    #locality sensitive hashing
    orders=[]
    candidates = []
    r = 4
    band = 128 // r;
    
    
    print("band : ",band)
    
    index = 0;
    for i in range(band):
      string = ''
      for j in range(r):
        string = string + str(minHash[i*r + j])
        hashvalue = hash(string)
        #print("string check :", string, "\n")
        #print("hashvalue : ", hashvalue)
      for j in range(size):
        string = ''
        for k in range(r):
          string = string + str(matrix[i*r + k][j])
          if(hash(string) == hashvalue):
            if reverseId[j] not in candidates:
              candidates.append( reverseId[j] );    
              print("candidate question found ", reverseId[j])
    #Jaccard Similarity
    for x in orders:
      list1=[]
      for y in range(128):
        list1.append(matrix[y][x]) 
      list2= orders
      intersection = len(list(set(list1).intersection(list2)))
      union = (len(list1) + len(list2)) - intersection
      sim = float(intersection / union)
      if sim > 0.01:
        sim=sim*100
        print("Sim= ",sim, " id: ",reverseId[x])
    
    #output
    score = 0;
    answerCount = 0
    viewCount = 0
    commentCount = 0
    for id in candidates:
      #print("candidate id : ",id)
      score += values[id][1]
      answerCount += values[id][2]
      viewCount += values[id][3]
      commentCount += values[id][4]
    length = len(candidates)
    if (length == 0):
        print ("Completely new question!")
    else:
      print ("Predicted score: ", score/length, "\n")
      print ("Predicted number of answers: ", answerCount/length, "\n")
      print ("Predicted view count: ", viewCount/length, "\n")
      print ("Predicted number of comments: ", commentCount/length, "\n")

java code method the sql
['java', 'code', 'method', 'sql']
final string :  ['java', 'code', 'method', 'sql']
index :  [47, 84, 473, 106]
band :  32
candidate question found  51780507
candidate question found  51167125
candidate question found  49328545
candidate question found  50427888
candidate question found  51090488
candidate question found  50744792
candidate question found  48397976
candidate question found  48965708
candidate question found  49510384
candidate question found  49146595
candidate question found  48532815
candidate question found  50299484
candidate question found  50750501
candidate question found  49331731
candidate question found  50353206
candidate question found  50821489
candidate question found  48746266
candidate question found  50089502
candidate question found  51985799
candidate question found  50827982
candidate question found  51777532
candidate question found  52024400
Predicted score:  0.2727272727272727 

Predicted number of answers:  1.59090909090